In [1]:
import pandas as pd
from datetime import datetime

In [2]:
swap_rates = pd.read_csv("./market-data-swap-rates.csv")
swap_rates.head(50)

,Date,Start Date,Tenor,Swap Rate
0,2021-01-13,2022-01-13,10y,1.081333
1,2021-01-13,2022-01-13,15y,1.265960
2,2021-01-13,2022-01-13,18M,0.221625
3,2021-01-13,2022-01-13,1y,0.195474
4,2021-01-13,2022-01-13,2y,0.279370
5,2021-01-13,2022-01-13,3y,0.404284
6,2021-01-13,2022-01-13,4y,0.536341
7,2021-01-13,2022-01-13,5y,0.664396
8,2021-01-13,2022-01-13,8y,0.950975
9,2021-01-13,2022-07-13,10y,1.155757


In [3]:
swap_rates.describe()

,Swap Rate
count,63180.000000
mean,2.426657
std,0.818694
min,0.167830
25%,1.718963
50%,2.592827
75%,3.036409
max,4.249516


train_data = swap_rates[swap_rates["Start Date"] < "2024-05-14"]
test_data = swap_rates[swap_rates["Start Date"] >= "2024-05-14"]

train_data.head(150)

train_data["Date"] = pd.to_datetime(train_data["Date"])
test_data["Start Date"]=pd.to_datetime(test_data["Start Date"])

train_grouped = train_data.groupby(['Start Date', 'Tenor'])
average_rate_train = grouped.mean()
average_rate_train

test_grouped = test_data.groupby(['Start Date', 'Tenor'])
average_rate_test = grouped.mean()
average_rate_test

outputpath='average_rate_train.csv'
average_rate_train.to_csv(outputpath, sep=',',index=True,header=True)

outputpath='average_rate_test.csv'
average_rate_test.to_csv(outputpath, sep=',',index=True,header=True)

grouped = swap_rates.groupby(['Start Date', 'Tenor'])
average_rate = grouped.mean()

def calculate_ema_group(group):
    group = group.copy()
    group['N'] = (group['Date'] - group['Date'].min()).dt.days + 1
    group['alpha'] = 2 / (group['N'] + 1)
    ema = [group.iloc[0]['Swap Rate']]
    for i in range(1, len(group)):
        alpha = group.iloc[i]['alpha']
        ema.append(alpha * group.iloc[i]['Swap Rate'] + (1 - alpha) * ema[-1])
    group['EMA'] = ema
    return group.drop(columns=['N', 'alpha'])

#swap_rates['EMA'] = swap_rates.groupby('Start Date')['Swap Rate'].transform(lambda x: calculate_ema(x, alpha))
#result_df = swap_rates.groupby('Start Date', group_keys=False).apply(lambda x: calculate_ema_group(x), include_groups=False)
#result_df = swap_rates.groupby('Start Date', group_keys=False).apply(calculate_ema_group, include_groups=False).reset_index(drop=True)
result_df = swap_rates.groupby('Start Date').apply(lambda group: calculate_ema_group(group)).reset_index(drop=True)


In [36]:
swap_rates["Date"] = pd.to_datetime(swap_rates["Date"])
swap_rates["Start Date"]=pd.to_datetime(swap_rates["Start Date"])

### EMA

In [4]:
swap_rates['N'] = swap_rates.groupby('Start Date')['Swap Rate'].transform('size')
swap_rates['alpha'] = 2 / (swap_rates['N'] + 1)

In [8]:
def calculate_ema(group):
    ema = [group.iloc[0]['Swap Rate']]  # EMA starts with the first data point
    alpha = group['alpha'].iloc[0]
    for price in group['Swap Rate'].iloc[1:]:
        ema.append(alpha * price + (1 - alpha) * ema[-1])
    return pd.Series(ema, index=group.index)

In [10]:
swap_rates['EMA'] = swap_rates.groupby('Start Date', group_keys=False).apply(calculate_ema,include_groups=False)

In [11]:
swap_rates

,Date,Start Date,Tenor,Swap Rate,N,alpha,EMA
0,2021-01-13,2022-01-13,10y,1.081333,9,0.2,1.081333
1,2021-01-13,2022-01-13,15y,1.265960,9,0.2,1.118258
2,2021-01-13,2022-01-13,18M,0.221625,9,0.2,0.938932
3,2021-01-13,2022-01-13,1y,0.195474,9,0.2,0.790240
4,2021-01-13,2022-01-13,2y,0.279370,9,0.2,0.688066
...,...,...,...,...,...,...,...
63175,2024-01-12,2039-01-12,2y,3.424505,9,0.2,3.259230
63176,2024-01-12,2039-01-12,3y,3.393758,9,0.2,3.286136
63177,2024-01-12,2039-01-12,4y,3.358897,9,0.2,3.300688
63178,2024-01-12,2039-01-12,5y,3.319531,9,0.2,3.304457


In [12]:
swap_rates.to_csv('swap_rates.csv', index=False)

### Reconcile with DummyTrade 1&2

In [16]:
dummyTrade1 = pd.read_csv("./dummyTrade1_data.csv")

In [18]:
dummyTrade1["Value Date"] = pd.to_datetime(dummyTrade1["Value Date"])
dummyTrade1["Expiry Date"]=pd.to_datetime(dummyTrade1["Expiry Date"])

In [21]:
merged_dummyTrade1 = pd.merge(dummyTrade1, swap_rates, how='inner', left_on=['Value Date', 'Tenor Bucket'], right_on=['Start Date', 'Tenor'])
merged_dummyTrade1 = merged_df.drop(columns=['Start Date'])

In [20]:
merged_df

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,Date,Start Date,Tenor,Swap Rate,N,alpha,EMA
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246,2021-03-02,2022-09-02,10y,1.554418,9,0.200000,1.554418
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341,2021-03-02,2022-09-02,10y,1.554418,9,0.200000,1.554418
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006,2021-03-02,2022-09-02,10y,1.554418,9,0.200000,1.554418
3,2022-09-02,dummyTrade1,USD,-10,-216872.430106,1y,2023-09-04,10y,4.333398,2021-03-02,2022-09-02,10y,1.554418,9,0.200000,1.554418
4,2022-09-02,dummyTrade1,USD,-5,-216146.310328,1y,2023-09-04,10y,5.679687,2021-03-02,2022-09-02,10y,1.554418,9,0.200000,1.554418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884515,2024-01-12,dummyTrade1,USD,50,-133255.494547,15y,2039-01-12,8y,-0.103813,2022-07-12,2024-01-12,8y,2.509168,18,0.105263,2.183681
884516,2024-01-12,dummyTrade1,USD,75,-121098.159209,15y,2039-01-12,8y,-0.104573,2022-01-12,2024-01-12,8y,1.672222,18,0.105263,1.666403
884517,2024-01-12,dummyTrade1,USD,75,-121098.159209,15y,2039-01-12,8y,-0.104573,2022-07-12,2024-01-12,8y,2.509168,18,0.105263,2.183681
884518,2024-01-12,dummyTrade1,USD,100,-108120.466797,15y,2039-01-12,8y,-0.038006,2022-01-12,2024-01-12,8y,1.672222,18,0.105263,1.666403


In [23]:
merged_dummyTrade1.to_csv('merged_dummyTrade1.csv', index=False)

In [24]:
dummyTrade2 = pd.read_csv("./dummyTrade2_data.csv")
dummyTrade2["Value Date"] = pd.to_datetime(dummyTrade2["Value Date"])
dummyTrade2["Expiry Date"]=pd.to_datetime(dummyTrade2["Expiry Date"])
merged_dummyTrade2 = pd.merge(dummyTrade2, swap_rates, how='inner', left_on=['Value Date', 'Tenor Bucket'], right_on=['Start Date', 'Tenor'])
merged_dummyTrade2 = merged_df.drop(columns=['Start Date'])
merged_dummyTrade2.to_csv('merged_dummyTrade2.csv', index=False)